In [ ]:
#r "nuget:Microsoft.Diagnostics.NETCore.Client,0.2.61701"
#r "nuget:Microsoft.Diagnostics.Tracing.TraceEvent,2.0.49"


using Microsoft.Diagnostics.NETCore.Client;
using System.Diagnostics;
using Microsoft.Diagnostics.Tracing;
using Microsoft.Diagnostics.Tracing.Parsers;
using Microsoft.Diagnostics.Tracing.Parsers.Clr;
using System.Threading.Tasks;
using System.Collections.Generic;
using System.Linq;
using XPlot.Plotly;

In [ ]:
// Put your process name here
var procName = "ConsoleApp";

var availableProcs = DiagnosticsClient.GetPublishedProcesses()
    .Select(x => Process.GetProcessById(x))
    .Select(p => new { name = p.ProcessName, id = p.Id, module = p.MainModule.FileName });

foreach (var p in availableProcs)
    display(p);

In [ ]:
var client = new DiagnosticsClient(availableProcs.Where(p => p.name == procName).FirstOrDefault()?.id ?? -1);
var session = client.StartEventPipeSession(
    providers: new List<EventPipeProvider> 
    { 
        new EventPipeProvider("Microsoft-Windows-DotNETRuntime", System.Diagnostics.Tracing.EventLevel.Verbose, (long)(-1))
    }
);

var eventDict = new Dictionary<string, List<TraceEvent>>();

var source = new EventPipeEventSource(session.EventStream);
Action<TraceEvent> processEvent = (TraceEvent obj) =>
{
    if (eventDict.TryGetValue(obj.EventName, out var eventList))
    {
        eventList.Add(obj.Clone());
    }
    else
    {
        eventDict.Add(obj.EventName, new List<TraceEvent> { obj.Clone() });
    }
};

var rundownParser = new ClrRundownTraceEventParser(source);
rundownParser.All += processEvent;
source.Clr.All += processEvent;

var readTask = Task.Run(() => 
{
    try
    {
        source.Process();
    }
    catch (Exception e)
    {
        display(e);
    }
});
    

Task.Delay(TimeSpan.FromSeconds(10)).Wait();
// source.StopProcessing();|
session.Stop();
display("Stopping");
readTask.Wait();
display("Stopped");

In [ ]:
foreach (var (key,val) in eventDict)
{
    display($"{key}, {val.Count}");
}

In [ ]:
Chart.Plot(
    new Graph.Bar()
    {
        y = eventDict.Values.Select(x => x.Count).ToArray(),
        x = eventDict.Keys
    }
)